# Dedicated Lane Simulation
In this notebook:
- We load our graph from a .gml file, complete with node and edge attributes.  
- We begin simulating the addition of a new network of dedicated fixed width lanes above our old one, in a few different cases
Our main variables:
- minimum car lane width (usually 3.5m, but we could try 7m for two lanes)
- dedicated lane width w
- km of dedicated lanes X

  
We create our subnetwork  with fixed car lane width (3.5m); once we pick a dedicated lane width w, we create the network keeping track of the total length of the streets (or fractional length) and checkpoint it various times.  
So for example we'll have the case were w= 2.5m (think of two-way bike lanes) and we'll have a network of X km total, but we'll also save the network at fractions of X. 
For each checkpoint we save the base network with updated edges and edge widths and bc, and node attributes, and save the subnetwork, while recalculating node attributes and edge bc.  

We fix car lane width: ideally a single lane is 3.5m, so we preserve at least one car lane per road, and test out different w for different X.  


When we save the networks we can save edgelist as a csv with attributes, and node attribute dict for nodes. I think this takes up less space than saving each graph in .gml

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import contextily as cx
import matplotlib.pyplot as plt
import networkx as nx
import igraph as ig
import json
import utils

In [ ]:
import os
cwd = os.getcwd()
graph_path = os.path.join(cwd, "MILANO/dataset_vehicles_preprocessed/base_network.gml")
base_G = nx.read_gml(graph_path)


In [ ]:
utils.plot_edges(base_G, save = False)

## Adding preferential bus lanes to the network


We decide a number of km X, known as our budget and create a network as close as possible to that length.  
To choose the edges, we rank them according to a score that keeps into account their width and their betweenness. 
we mix width and betweenness with a linear combination.  
Width and betweenness are not directly comparable, so we must transform our data in some way. Since we're interested in creating a score to rank our networks, we rank each edge according to betweenness and width respectively. from there, we define the edge's p-value as  
$1-rank/N$  
where N is the number of edges in the network. This is a p-value because it is the fraction of edges ranked higher than the edge in question, which corresponds to the probability of sampling an edge with a higher ranking assuming the data's distribution is the observed on. This is the definition of a p-value.
z-score computation happens before simulations begin, whereas final scores depend on the value of $\alpha$ which is specific
to the simulation.  
After having computed s, edges are ranked in s, and simulations simply become a selection of the highest ranking edges until budget is satisfied. 

In [ ]:
utils.list_attributes(base_G)

In [ ]:
# Apply the function to base_G
utils.p_values(base_G)

In [ ]:
def dbl_network_mixed(G, w, budget, alpha = 0.5, destructive = False, car_lane = 3.5):
    #takes base network and performs simulation with mixed strategy.
    #w is the width of the dedicated lane in metres
    #budget is the total length of dedicated lane in km
    # alpha is the mixing coefficient
    #destructive = False means that base network edges can not be removed, and must be wide enough to have at least one car lane left.


    G = G.copy()
    threshold = w + car_lane
    if destructive:
        threshold = w
    budget = budget*1000 #convert from km to m

#create score ranking of edges
    edgelist_base = nx.to_pandas_edgelist(G, edge_key = 'key')
    edgelist = edgelist_base[edgelist_base.width > threshold].copy()#exclude edges that are too narrow
    edgelist['score'] = alpha * edgelist['p_w'] + (1 - alpha) * edgelist['p_bc']
    edgelist = edgelist.sort_values('score', ascending = False)
    edgelist['old_index'] = edgelist.index #keep track of original index
    edgelist = edgelist.reset_index(drop = True) #reset index to avoid issues with cumsum
#respect budget
    edgelist['cumsum'] = edgelist['length_for_bc'].cumsum()
    budget_idx = edgelist[edgelist['cumsum'] > budget].index.min()
    if pd.isna(budget_idx): #ugly exception handling but it's ok for now...
        budget_idx = edgelist.index[-1]
    if budget_idx is not None:
        edgelist = edgelist.iloc[:budget_idx + 1] #include the first edge that passes the budget


#CREATE SUBGRAPH
    edgelist_iterable =  set(map(tuple, edgelist[['source','target','key']].values)) 
    sub_G = G.edge_subgraph(edgelist_iterable).copy()

#UPDATE WIDTHS
    # sub
    w_dict_sub = {edge: w for edge in sub_G.edges(keys = True)}
    nx.set_edge_attributes(sub_G, w_dict_sub, 'width') #assign width to subgraph
    # base

    #make the index of edgelist its old index column
    edgelist.sort_values('old_index', inplace=True) #re-sort by original index
    edgelist.set_index('old_index', inplace=True)
    edgelist.width = w
    temp = edgelist.width.reindex(edgelist_base.index, fill_value=0)
    edgelist_base['width'] = edgelist_base['width'] - temp
    attr = list(list(G.edges(data=True))[0][-1].keys())
    G = nx.from_pandas_edgelist(edgelist_base, edge_attr = attr,create_using=nx.MultiGraph)

    return G, sub_G

## Pseudocode for simulation Algorithm

1. **Set width threshold**  
   `threshold = w + min_car_lane`
    We use `w = 2.5m` and `min_car_lane = 3.5m` in all simulations in this work.
2. **Filter edges**  
   - Keep only edges with width greater than the threshold.

3. **Compute edge scores**  
   - For each edge:  
     `score = alpha * R_w + (1 - alpha) * R_b`

4. **Sort edges by score (descending)**

5. **Select edges within budget**  
    - compute cumulative sum of lengths of edges
    - Find the first edge where cumulative sum exceeds the budget.  
    - Select all edges up to and including this edge.

6. **Create subgraph from selected edges**
    - This is the subgraph of bike lanes
7. **Update widths in subgraph**  
    - Set all widths of subgraph to `w`.

8. **Update widths in base graph**  
    - For each affected edge in base graph, subtract `w` from its width.
9. **Return updated base graph and subgraph**

### Single iteration


So a single execution of the simulation starts from the base network, calls dbl_network to generate two new networks, updates node and edge attributes of both, then saves node attributes and edge attributes of both, in two dicts or two .csv files (saved from dataframes)

In [ ]:
cwd

In [ ]:
result_path = ''
save_path = os.path.join(result_path, "simulations")
graph_path = os.path.join(result_path, "MILANO/dataset_vehicles_preprocessed/base_network.gml")
# make sure the save path exists
if not os.path.exists(save_path):
    os.makedirs(save_path)
    
base_G = nx.read_gml(graph_path)
utils.p_values(base_G)

In [ ]:
w = 2.5
budget = 2
alpha = 0
destructive = False
d = 'non_destructive' if not destructive else 'destructive'
car_lane = 3.5
G1, G2 = utils.dbl_network_mixed(base_G, w, budget, alpha, destructive, car_lane)
utils.update_and_save(G1, G2, w, budget, alpha, d, car_lane, save_path = save_path)


## Loop: 
This is the loop used to create the networks examined in Basilone et. al (2025). You can modify the parameter ranges as you wish

In [ ]:
# NB this loop takes hours to run. 

import time
start_time = time.time()
save_path = os.path.join(result_path, "simulations")
try:
    w = 2.5
    alpha_list = [0, 0.1, 0.3, 0.5, 0.7, 0.9, 1.0]
    budget_list = list(np.arange(1,20,1))+ list(np.arange(20,200,10))+ list(np.arange(200, 1000, 40))
    # budget_list =  list(np.arange(20,200,10))+ list(np.arange(200, 1000, 40))
    
    destructive_list = [False]
    car_lane_list = [3.5]
    for car_lane in car_lane_list:
      for destructive in destructive_list:
            for alpha in alpha_list:
               print('simulating at alpha = ', alpha, 'w =', w)
               for budget in budget_list:
                  d = 'non_destructive' if not destructive else 'destructive'
                  G1, G2 = utils.dbl_network_mixed(base_G, w, budget, alpha, destructive, car_lane)
                  utils.update_and_save(G1,G2,w,budget, alpha,d,car_lane, save_path = save_path)
                  print('saved simulation for alpha = ', alpha, 'w =', w, 'budget =', budget)
    pass
finally:
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time:.2f} seconds")

### Loading data:
an example to see if the simulations were executed correctly

In [ ]:
w = 2.5
alpha = 0
budget = 2 
strategy = 'bc' 
destructive = False 
d = 'non_destructive' if not destructive else 'destructive'
car_lane = 3.5
base_or_sub = 'baseG'
load_path = f'simulations/car_lane={car_lane}/{d}/w={w}/alpha={alpha}'

In [ ]:
base_G = utils.load_data(load_path, base_or_sub, budget)